<a href="https://colab.research.google.com/github/wangbxj1234/offset_pt_try/blob/main/%E5%8D%95%E7%82%B9fps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import torch

In [18]:
b=np.random.rand(2048,3)#########set from numpy
instance = torch.Tensor(b)
def farthest_point_sample_1(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    device = xyz.device
    N, C = xyz.shape#############9840, 2048, 3
    centroids = torch.zeros(npoint, dtype=torch.long).to(device)#(9840, 1024)
    distance = torch.ones(N).to(device) * 1e10        ###########(9840, 2048)
    farthest = torch.randint(0, N,(1,), dtype=torch.long).to(device)######low:0,high:2048,  (9840,1)
    #batch_indices = torch.arange(B, dtype=torch.long).to(device)########### arange: to generate (0,1,2,.....,9839)
    for i in range(npoint):
        centroids[i] = farthest##################[1]
        centroid = xyz[farthest, :].view(1, 3) ######(9840,2048,3)
        dist = torch.sum((xyz - centroid) ** 2, -1)##################(9840,2048)
        distance = torch.min(distance, dist)###############NO LARGER THAN 1e10
        farthest = torch.max(distance, -1)[1]############## 取最远距离的那个点 得到坐标
    return centroids
fps_idx_single = farthest_point_sample_1(instance, 1024)
fps_idx_single.shape

torch.Size([1024])

In [21]:
def index_points_1(instance, idx):
    """
    Input:
        instance: input points data, [ N, C]
        idx: sample index data, [ S, [K]]
    Return:
        new_points:, indexed points data, [ S, [K], C]
    """
    raw_size = idx.size()###s
    idx = idx.reshape(-1)
    res = torch.gather(instance, 0, idx[..., None].expand(-1, instance.size(-1)))#先把idx加上dim=3的第二维, 再用gather把instance中的对应点按索引提取出来。
    #### 1024 -》（1024,3），然后从（2048，3）中提取对应的点。
    return res.reshape(*raw_size, -1)####就是idx的shape（2维或者三维），再加上最后一维c，理论上这个-1也可以写成'instance.size(-1)'吧。

In [22]:
torch.cuda.empty_cache()#####清除显存
pointcloud = index_points_1(instance, fps_idx_single)########### 按照index 返回 3维点。
pointcloud.shape

torch.Size([1024, 3])